# Gender differences in parliamentary speeches
Background

Available through API with XML files, but also as csv files.



In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [4]:
speeches_df = pd.read_csv('speeches_201415.csv')

In [5]:
speeches_df.head()

,dok_id,dok_rm,dok_nummer,dok_datum,avsnittsrubrik,kammaraktivitet,anforande_nummer,talare,parti,anforandetext,intressent_id,rel_dok_id,replik
0,H2091,2014/15,1,2014-09-29T00:00:00Z,Hälsningsanförande från ålderspresidenten,NaN,1,Per Westerberg (M),M,Hälsningsanförande från ålderspresidenten Ära...,031005479013,NaN,N
1,H2091,2014/15,1,2014-09-29T00:00:00Z,Val av talman,val,2,Mikael Damberg (S),S,Herr ålderspresident! Att vara talman i Sverig...,014744660015,NaN,N
2,H2091,2014/15,1,2014-09-29T00:00:00Z,Val av talman,val,3,Anna Kinberg Batra (M),M,Herr ålderspresident! Ärade ledamöter av Sveri...,012053331219,NaN,N
3,H2091,2014/15,1,2014-09-29T00:00:00Z,Val av förste vice talman,val,4,Anna Kinberg Batra (M),M,Herr ålderspresident! Som företrädare för Alli...,012053331219,NaN,N
4,H2091,2014/15,1,2014-09-29T00:00:00Z,Val av andre vice talman,val,5,Mattias Karlsson (SD),SD,Val av andre vice talman Herr ålderspresident...,0831636664429,NaN,N
